In [1]:
import tweepy
import re
import pandas as pd
import numpy as np
from textblob import TextBlob

In [8]:
consumer_key = "yJI7VztRV1IFd52g5Qm6CoQ4k"   
consumer_secret = "yN2z5rlWziN9Wdi9lB9XjLvogl5ItqXoBlcvck8S2N7mj3FtfF"   
access_token = "909527697646395393-HUEr0Aws7DCnn3VWGrWnMakmPY0SlAI"
access_token_secret = "4ntwDjAKihF9ZKELxMaxRc4GqMd1A9yAamlbmFF7kjCtE"


In [9]:
# Tweepy settings
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)   
auth.set_access_token(access_token, access_token_secret)   
api = tweepy.API(auth)   


In [23]:
# try full archive

tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='development', query='vegan', fromDate='"201008130000"').items(100)
for tweet in tweets:     
    print(tweet.text, tweet.created_at) 

TweepError: {'message': 'Forbidden: Authentication succeeded but account is not authorized to access this resource.', 'sent': '2021-04-08T13:43:48+00:00', 'transactionId': '00de88c3000ecbfb'}

In [15]:
# basic timeline for Anran's account
public_tweets = api.home_timeline()
for tweet in public_tweets:     
    print(tweet.text, tweet.created_at) 

RT @L_Brierley: 🎓Interested in a #mastersdegree #MSC in #stats and #datascience for #health?

Next week is Postgrad Open Week @LivUni @livu… 2021-04-08 13:24:59
How to behave in a commercial gallery, if you’ve never dared set foot in one https://t.co/bnJMq2OkkB 2021-04-08 13:15:00
Do you work for an England-based #charity that's undergone a successful cultural change in the way your organisatio… https://t.co/BH7PK5dZxa 2021-04-08 13:10:02
✨ Shoot your shot.

📷 ‘Redefining Ballerism: Upping the Ante’, 2009 by Seattle-based metal artist #JanaBrevick… https://t.co/LQ14hBkpu8 2021-04-08 13:07:31
RT @MLBrook: and its not like we lack for great orgs working on data at the local level either... cc @opendatamcr @ODILeeds 2021-04-08 12:45:53
The biannual “Taking Stock of Teens,” report, which surveys over 7,000 teens showed what brands and retailers can e… https://t.co/HkxRN7OU4V 2021-04-08 12:34:00
The artist Frederic William Burton, who was born in County Wicklow #OTD in 1816, may be little k

In [16]:
# Get the User object 
user = api.get_user('AnranZhao')
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
   print(friend.screen_name)

AnranZhao
17
UoM_ITS
BeveragesHot
SorchaUKDS
GillUKDS
CMI_Man
Patty_NZ
VanessaAHiggins
ABloom_20
UKDSImpact
mikybartoli
JKasmireComplex
_JAStark
PyDataMCR
JosephAllen1234
opendatamcr
UKDataService
stephen_whitemn
McrContemporary
OfficialUoM
arthistsoc_uom


In [17]:
# collect 1000 English tweets on 'vegan' and show the full text
# but all retweets are not full 
for status in tweepy.Cursor(api.search, q='vegan', lang='en', tweet_mode='extended' ).items(1000):
    print (status.full_text)

@JeremyCobbold @juliana_monty @ECOAlphaTango @themankumar @angie_karan @Ivy_MiddletonUK @trev_vegan @_AnimalAdvocate @joelle47019650 @Veganella_ @GretaThunberg @hilltopgina @ItsMadhuriJain I don’t feel it is a political issue. It is a moral issue. Warriors on both sides of the political spectrum.
RT @IamLivingPhenom: Introducing Living Phenom’s Maple Hemp Melk shake.
#Organic #HealthyRecipes #Nutrition #PlantBased #Vegan #GlutenFree…
RT @thefeelgoodcafe: Such a pretty #MatchaLatte 😍! #vegan #latte #veganmilk #oatly #oatmilk #oatlatte #oatlylatte https://t.co/PhGV3JNNeb h…
RT @VeganShoesLon: #eamonnholmes apparently im told was saying today on TV those that work with animals seem to be the happiest, did he mea…
RT @IamLivingPhenom: Introducing Living Phenom’s Maple Hemp Melk shake.
#Organic #HealthyRecipes #Nutrition #PlantBased #Vegan #GlutenFree…
RT @thevegansnuts: If your mother doesn't Moo it's NOT for you! Drink plant milk it's better tasting and cruelty free too. No disgusting pu…

RT @molesuk: Read more here, and go #Vegan https://t.co/3g5LyUwz1R
Trader Joe’s is getting a vegan version of those mini hold the cone things
RT @KatLikesSykes: I wanna be vegan starting next year ooh
RT @KatLikesSykes: I wanna be vegan starting next year ooh
If I were to put up a vegan resto, I’ll probably focus on plant based pasta 🍝 

... coz I love plant based pasta 🤤
Gonna start a fire with your vegan ass you to skinny lol bone on bone out here twerking 😂😂😂😂
@FunkyWrite @fookinright @jillieduke @MsHeath4 @borchard76 @Average_Dad1 "may provide health benefits in the prevention and treatment of certain diseases."

This does not = vegan diet is safe for everyone.
RT @heavenskincare: New product launch! I’m so excited to unveil my latest vegan cream, designed to help those who suffer from sensitive sk…
RT @molesuk: Read more here, and go #Vegan https://t.co/3g5LyUwz1R
RT @Remember_Phae: Hear me out : Netflix and Chill'd, oatly choccy milk and 1 ba nae nae , blend and you'll have the b

RT @danblock4: AH could eat a fucking hamburger and five minutes later claim to be vegan.
Your A-to-Z guide to cruelty free, vegan makeup and skincare brands #vegan #news https://t.co/Z6elPoKcyM
🌟🌟Buy Now🌟🌟
.
Buy now from our new collection of Vegan Belts &amp; Wallets, The JP Series. Named after our beloved Joaquin Phoenix.
.
Link in Bio or visit our website https://t.co/CfBV4brbIR @ London, Unιted Kingdom https://t.co/G1WbAfthB6
Don't eat meat but love jerky? Not a problem! We have vegan jerky for the jerky lovers who don't eat meat!
https://t.co/DYx6Y6tafq https://t.co/3sWq7JRRAG
RT @FrasierHarry: The 1918-1919 pandemic, which started on a Kansas chicken farm, killed more, in a few months, than the whole of World War…
RT @Veganella_: This character on the right represents the cognitive dissonance of millions of people, and they just don’t realise it. 

Pl…
forgot to post last week but !! 8 years since i stopped eating meat ++ this is how much ive saved (first one is when i was veget

RT @Unproblemattic: An Anti-Vaxxer and a Vegan walk into a bar. Who tries to convert you first?
RT @MeatFreeMonday: Yum! Look at this delicious #vegan Carrot Cake recipe from @guardian! 🥕
https://t.co/QlQn3hVYBV
It's way past time to make the donuts vegan. https://t.co/25xq2akxWz
@redsteeze "I sat here sobbing into my lunch of vegan latte, kale chips and tree bark. Here I am, just back from the BDS rally, hoping to be comforted by NPR, only to be confronted by *gasp* facts! Is it possible, if you MUST run factual stories, to at least apologize for existing?"
Be #Kind and Be #Vegan for a good reason.Saving an #Animals life. https://t.co/MZKPRiWxVO
@davyay Oh I get it! When I was vegetarian I felt the same way about meat-free food made to be like meat. Like, do I want 'vegan chicken kiev'? No, dude just no
RT @TweetsPrudhvi: A calf drinking their mother's milk is bad for the profits of the dairy industry. The baby is therefore taken away from…
RT @Unproblemattic: An Anti-Vaxxer and a Veg

RT @tyrathetaurus: it takes 2 seconds to retweet my NEW vegan munchie treats for 4/20🔥
inspired by our fav childhood cereals!
(black owned…
RT @thatvegangirl_: my vegan Bang Bang Shrimp recipe and tutorial just went up on That Vegan Girl!

per usual, Link in Bio! 

Let me know w…
RT @milkthecashews: YOU ARE STILL VEGAN IF YOU GET THE COVID VACCINE DON’T LET ANYONE TELL YOU DIFFERENT
When it comes to good nutrition, more is better. JuicePlus Berry Blend has farm-fresh ingredients providing added nutrition from a wide variety of berries. #Buy 👉 https://t.co/mKF6gmg5fN #selfcare #vegan #plantbased #plantbased #PlantBasedDiet #ThursdayThoughts #thursdaytreat https://t.co/mOpzWDdCX7
@19D_Vegan @barb_avon @NaomiWildman3 Thank you, I appreciate your kind words so much! 

I do love the versatility on Instagram that allows me to try and share those!
A vegan is a person who has the crazy idea that animals shouldn't suffer #endspeciesism
RT @vipmumsndads: Just because...you love it @youlovegifts 

...

RT @TweetsPrudhvi: They remain oppressed from birth till the time they are “spent” and see death, all so that we can exploit them for food,…
RT @vishnusai11: Animals are here to live with us
Once we see the horrors of the meat industry it isn't easy to hide our hypocrisy.
#govega…
@traceybib @FightyWoman @alihenderson10 @ghud68 My mum and dad are confirmed meat eaters but enjoy what I take them and have asked me to get them some vegan bits as well
Following are some health benefits of following a vegan diet.
https://t.co/3V0EEsR9pR
vegan-diet
#VeganDiet #HealthBenefit #UrgentCare
We don't need milk from another animal to fulfill our nutritional requirements. Ditch dairy. Ditch cruelty. Go vegan! @NewsX #govegan #endspeciesism
RT @kevinveegan: “Vegans have no effect on anything,” the fragile man screeched, as the vegan had the immediate effect of making him demons…
RT @Aditya98968582: India is failing its consitutional values of justice and equality when it does not consider the interes

RT @NaturaEmporium: Take a peek at all the wonderful #natural &amp; #Vegan #selfcare products you'll find at https://t.co/pdjCApw5iS using only…
Teriyaki “wings” 😍⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀
So many different ways to prepare these guys. Comment below your favorite marinade! https://t.co/6vvU08Vyie
“When we suffer, we suffer as equals. And in their capacity to suffer, a dog is a pig, is a bear, is a boy.” - Philip Wollen Choose justice for all animals. Go vegan! @PujyaNamramuni  #govegan #endspeciesism
RT @ThePumpkinPatc5: Happy release day! Springtime is here, mother's day is coming and I have got the goods! 💚 All vegan and handmade, soap…
RT @becca9893: Hi, if you’re seeing this and are vegetarian/vegan can you DM me why you decided to make this choice. And if you’re reading…
Demand for milk, meat, leather, wool, etc. is making people to repeatedly make animals pregnant and reproduce. Animals eat vegetables, grains, legumes, soy, etc. This food could be given to the hungry poor people. Become

RT @heavenskincare: New product launch! I’m so excited to unveil my latest vegan cream, designed to help those who suffer from sensitive sk…
RT @smitajs: We are not doing animals any favour by going vegan. We are just "Being Human" in true sense. Find the lost humanity. #GoVegan…
RT @claudelynnleger: What the heck @Mrs_Veg I’m also posting a bread. Raisin cinnamon. #vegan #veganhour #VeganRecipeHour #sourdough https:…
RT @aakashzure: Whenever we eat animal products, we pay for the abuse and oppression of animals. Leave animal products off your plate. Go v…
Humans have created animal based industries for food, clothing, experimentation, entertainment, etc. We have selectively bred their bodies and controlled their reproductive systems. End this cruelty. Go vegan.@News18Kannada @DDChandanaNews  #govegan #endspeciesism
Humans have created animal based industries for food, clothing, experimentation, entertainment, etc. We have selectively bred their bodies and controlled their reproductive

Why doesn’t the @coopuk like having large shops? You have released so many new vegan treats, yet all the ones near us are small and don’t do anything good
End human supremacy!
Go vegan!
#GoVegan #endspeciesism
RT @RuthSeq: #Coffee twitter, what are your favourite #vegan mylks?
Once I make vegan spicy aioli it’s OVER for u bitches
My daughter tells me that a vegan fish cake is a "vish cake" and there's somewhere in Levenshulme that sells them. Does my local Green Party candidate know this? https://t.co/gwBNXZv5jk
Clearly calling things plant based versus vegan polls better in focus groups.

Plant based *feels* better than vegan. 

Because otherwise why would you call something that’s main ingredients are apples and flour plant based?? https://t.co/KpnUyASPX9
RT @SummerYuleRDN: Tempeh Bolognese Sauce Recipe (even tempeh haters love it!) 🍝

Get the recipe: https://t.co/eE52qWJwUk 

#healthyrecipes…
@Gabby_Kam_J Hey! I'm Becky. I make vegan and cruelty free lipgloss and wax melts, aswell a

RT @AjayKumar669: https://t.co/CyRXEAN5t4 💕Be #vegan for them🐖, not for #yourself.😊His place is not in the plate😔.If you can feel their pai…
@kevinstock12 Looks like a vegan thou. All meateaters i know are nice to look at
@Plant_proof @kevinnbass @ahoysvet @MBPowerProject I get it, but it's hard on Twitter replacing 'vegan', often loaded, with longer words! I wish vegan activists (&amp; I am one too, but focusing on far more than just one issue) would accept the suggestion to use vegan as meaning just ''eating only plants' https://t.co/SWzphzRkTE
RT @VeggieGalaxy: New Blue Plate Special starts tomorrow: JALAPEÑO POPPER-STUFFED BURGER!
House-made Galaxy burger stuffed w/ jalapeño popp…
@Jen80376375 @TheVeganSociety @OurFutureNormal Agreed. The Vegan Society sure are epic! If you ever want any help, support or guidance taking the full step - feel free to reach out 🙏💚
RT @spellcastersim: I can’t deal with this hair colour in The Sims 4 like it makes me so mad. Like is anyone using this ha

In [18]:
# collect 200 tweets from user TheVeganRD and show the full text
# NB, here retweets lost the identification 'RT' and become common tweets

results = api.user_timeline(id='TheVeganRD', count=200, tweet_mode='extended')
for status in results:
    try:
        print(status.retweeted_status.full_text)
    except AttributeError:  # Not a Retweet
        print(status.full_text)

@Mjwenckus Some combination of salt, fat, flavor, texture, umami. After all, the early nondairy cheeses contained casein and they were anything but addictive!
I've been writing vegan advocacy material since the early 1990s. I'm totally disgusted by the lack of credibility in some of the most oft-repeated claims, just as I know my friend @TheVeganRD has equal despair over bogus vegan health claims. 5/
If someone says you can lose weight without being on a "diet" by eating plant-based and then they say you can never have oil or sugar or veggie meats or refined pasta--guess what? They’re putting you on a diet.
@SuchFun_AreWe Agree -- I am a very big fan of Violife! It's on sale right now and my refrigerator is packed with it.
@mdryja I just looked it up -- and now it's on my list for the next time I'm in southern CA.
@tempehlover @moozoogabba @SuchFun_AreWe That cheese looks amazing!
@NCanimalvoters @jan_dutkiewicz Oh, those corn dogs!
Such an interesting article on the history and evolut

In [19]:
# comparison
results = api.user_timeline(id='TheVeganRD', count=200, tweet_mode='extended')
for status in results:
    print(status.full_text)

@Mjwenckus Some combination of salt, fat, flavor, texture, umami. After all, the early nondairy cheeses contained casein and they were anything but addictive!
RT @vegan: I've been writing vegan advocacy material since the early 1990s. I'm totally disgusted by the lack of credibility in some of the…
If someone says you can lose weight without being on a "diet" by eating plant-based and then they say you can never have oil or sugar or veggie meats or refined pasta--guess what? They’re putting you on a diet.
@SuchFun_AreWe Agree -- I am a very big fan of Violife! It's on sale right now and my refrigerator is packed with it.
@mdryja I just looked it up -- and now it's on my list for the next time I'm in southern CA.
@tempehlover @moozoogabba @SuchFun_AreWe That cheese looks amazing!
@NCanimalvoters @jan_dutkiewicz Oh, those corn dogs!
Such an interesting article on the history and evolution of #vegan cheese. (Just skip over the questionable claims about "addictive" properties of casein.)  

In [22]:
# test: without cursor, only tweets on the first result page is shown, which is 100 tweets here
count=0
res=api.search(q='veganism', lang='en',tweet_mode='extended', count=100000)
for tweet in res:
    count+=1
    print(tweet.full_text)
    print(count)

Veganism is a social justice movement. The way we fight for slavery, racism, women rights... now is time to fight for animals. #GoVegan #endspeciesism
1
Veganism &amp; spirituality do not go hand and hand. Argue with yo mama while I eat my cheese &amp; chicken nuggies. https://t.co/BDM4W4y7tF
2
@jewish_activist the way most leftists talk about animal rights is on par with how ben shapiro “debates”. for all the complaints about “white veganism” they sure love using BIPOC people as shields for their own shitty habits
3
RT @i_am_RAGHAV: There is no argument against veganism. Stop being a plant rights activist. Stop the slaughter, exploitation, enslavement,…
4
@makeyallsweat @Klanderverdig @milkthecashews Firstly, I have clarified several times, vegans have contracted the virus. Secondly, I stand by my choice and the right to choose without being judged. Thirdly, you say veganism can't fix it, when the science says Covid-19 jumped from animals to humans. Same as HIV, SARS etc.
5
RT @Veggie

In [33]:
# store only text and time

df=pd.DataFrame()
content=[]
time=[]

search=api.search(q='COVID', lang='en', count=5,tweet_mode='extended')
for tweet in search:
    content.append(tweet.full_text)
    time.append(tweet.created_at )
    
df['content']=content
df['time']=time
df

,content,time
0,RT @PChidambaram_IN: The Centre is targeting M...,2021-04-08 13:55:22
1,RT @ShapedInternet: It's been a year since we ...,2021-04-08 13:55:22
2,RT @ArshiyaMuskaan7: Respected @DrTamilisaiGuv...,2021-04-08 13:55:22
3,RT @WebMD: The Pfizer and Moderna COVID-19 vac...,2021-04-08 13:55:21
4,RT @PoliticsForAlI: I would be more likely to ...,2021-04-08 13:55:21


In [29]:
# store all features
# ref: from 26min, https://www.youtube.com/watch?v=dvAurfBB6Jk&t=2637s&ab_channel=CodingEntrepreneurs 
# need further editing. what's difference between author and user?


search=api.search(q='vegan', lang='en', count=5,tweet_mode='extended')

columns = set()
allowed_types = [str, int]
tweets_data = []
for status in search:
    status_dict = dict(vars(status))
    keys = status_dict.keys()
    single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
    for k in keys:
        try:
            v_type = type(status_dict[k])
        except:
            v_type = None
        if v_type != None:
            if v_type in allowed_types:
                single_tweet_data[k] = status_dict[k]
                columns.add(k)
    tweets_data.append(single_tweet_data)

        
header_cols = list(columns)
header_cols.append("user")
header_cols.append('author')

df = pd.DataFrame(tweets_data, columns=header_cols)
df

,in_reply_to_user_id_str,retweet_count,id,in_reply_to_status_id_str,in_reply_to_screen_name,id_str,in_reply_to_user_id,source,full_text,favorite_count,in_reply_to_status_id,lang,source_url,user,author
0,NaN,6,1380156533716226056,NaN,NaN,1380156533716226056,NaN,Twitter for iPhone,RT @Unproblemattic: An Anti-Vaxxer and a Vegan...,0,NaN,en,http://twitter.com/download/iphone,CallmeQueen5,CallmeQueen5
1,NaN,0,1380156520671948800,NaN,NaN,1380156520671948800,NaN,Twitter Web App,vegan greek yoghurt is so good,0,NaN,en,https://mobile.twitter.com,doubledepress0,doubledepress0
2,NaN,58,1380156508994998275,NaN,NaN,1380156508994998275,NaN,Twitter for Android,RT @KetoAurelius: To be at optimal health on a...,0,NaN,en,http://twitter.com/download/android,CopyOfOne,CopyOfOne
3,NaN,0,1380156493543133191,NaN,NaN,1380156493543133191,NaN,IFTTT,Beyond Cabbage: How to Spice Up Your Sauerkrau...,0,NaN,en,https://ifttt.com,PlantBasedGent,PlantBasedGent
4,327187863,0,1380156476367515658,1380149382671167496,Wxcafe,1380156476367515658,327187863.0,Twitter Web App,@Wxcafe it’s not too hard to forge a vegan ID.,0,1.380149e+18,en,https://mobile.twitter.com,Q_quiscula,Q_quiscula


In [33]:
df.loc[0].content

'RT @MSNBC: WATCH: President Biden urges Americans to continue following Covid-19 guidelines: "Mask up, it’s a patriotic duty. It’s the only…'